In [1]:
from prophet import Prophet
import pandas as pd
import numpy as np

In [2]:
def get_stan_init(m):
    """Retrieve parameters from a trained model.

    Retrieve parameters from a trained model in the format
    used to initialize a new Stan model.

    Parameters
    ----------
    m: A trained model of the Prophet class.

    Returns
    -------
    A Dictionary containing retrieved parameters of m.
    """
    res = {}
    for pname in ['k', 'm', 'sigma_obs']:
        if m.mcmc_samples == 0:
            res[pname] = round(m.params[pname][0][0], 4)
        else:
            res[pname] = np.mean(m.params[pname]).round(4)
    for pname in ['delta', 'beta']:
        if m.mcmc_samples == 0:
            res[pname] = m.params[pname][0].round(4)
        else:
            res[pname] = np.mean(m.params[pname], axis=0).round(4)
    return res

In [3]:
df = pd.read_csv('../examples/example_wp_log_peyton_manning.csv')
df1 = df.loc[df['ds'] < '2016-01-19', :]  # All data except the last day
m1 = Prophet().fit(df1) # A model fit to all data except the last day

INFO:cmdstanpy:Chain [1] start processing
INFO:cmdstanpy:Chain [1] done processing


In [4]:
m1_params = get_stan_init(m1)

In [5]:
m1_params

{'k': -0.3567,
 'm': 0.6259,
 'sigma_obs': 0.0376,
 'delta': array([-0.    ,  0.    ,  0.3413,  0.4757, -0.    , -0.0032, -0.2278,
        -0.2729,  0.    ,  0.    , -0.    ,  0.3028,  0.2135,  0.0101,
         0.    , -0.8595, -0.    , -0.    , -0.    ,  0.4698,  0.0081,
         0.    , -0.3406, -0.    ,  0.    ]),
 'beta': array([-0.03  ,  0.0482,  0.01  , -0.0023,  0.0125,  0.0156,  0.0095,
         0.0008,  0.0039, -0.0059,  0.0075, -0.0052,  0.0021, -0.0048,
        -0.0041, -0.0077, -0.0008,  0.0013,  0.0019,  0.007 , -0.0123,
        -0.0106,  0.0094,  0.0085,  0.0009, -0.0035])}

In [6]:
m2 = Prophet()
m2 = m2.fit(df, init=m1_params)

INFO:cmdstanpy:Chain [1] start processing
INFO:cmdstanpy:Chain [1] done processing


In [26]:
get_stan_init(m2)

{'k': -0.3562,
 'm': 0.6259,
 'sigma_obs': 0.0376,
 'delta': array([ 0.000e+00, -0.000e+00,  3.413e-01,  4.758e-01, -0.000e+00,
        -2.600e-03, -2.279e-01, -2.728e-01, -0.000e+00,  0.000e+00,
        -0.000e+00,  3.014e-01,  2.127e-01,  9.200e-03,  1.000e-04,
        -8.615e-01, -0.000e+00, -0.000e+00,  0.000e+00,  4.716e-01,
         9.200e-03, -0.000e+00, -3.376e-01, -0.000e+00,  0.000e+00]),
 'beta': array([-0.03  ,  0.0482,  0.01  , -0.0023,  0.0125,  0.0156,  0.0095,
         0.0008,  0.0039, -0.0059,  0.0075, -0.0052,  0.0021, -0.0048,
        -0.0041, -0.0078, -0.0008,  0.0013,  0.0019,  0.007 , -0.0123,
        -0.0106,  0.0094,  0.0085,  0.0009, -0.0035])}

In [9]:
from pathlib import Path

In [10]:
DATA = pd.read_csv(
    Path("..") / "python" / "prophet" / "tests" / "data.csv",
    parse_dates=['ds'],
)

In [29]:
def test_fit_warm_start():
    previous_df = DATA.iloc[:500]
    df = DATA.iloc[:510]
    m = Prophet()
    m = m.fit(previous_df)
    m_params = get_stan_init(m)
    m2 = Prophet()
    m2 = m2.fit(df, init=m_params)
    return m2

def test_sampling_warm_start():
    previous_df = DATA.iloc[:500]
    df = DATA.iloc[:510]
    m = Prophet(mcmc_samples=100)
    m = m.fit(previous_df, show_progress=False)
    m_params = get_stan_init(m)
    m2 = Prophet(mcmc_samples=100)
    m2 = m2.fit(df, init=m_params, show_progress=False)
    return m2

In [11]:
previous_df = DATA.iloc[:500]
df = DATA.iloc[:510]
m = Prophet()
m = m.fit(previous_df)
m_params = get_stan_init(m)

INFO:cmdstanpy:Chain [1] start processing
INFO:cmdstanpy:Chain [1] done processing


In [12]:
m_params

{'k': -0.9754,
 'm': 0.4568,
 'sigma_obs': 0.0501,
 'delta': array([ 0.000e+00,  0.000e+00,  0.000e+00,  3.765e-01,  7.350e-01,
         3.893e-01,  2.000e-04,  0.000e+00, -0.000e+00, -2.778e-01,
        -3.480e-01, -0.000e+00,  0.000e+00,  0.000e+00, -0.000e+00,
         5.647e-01,  9.929e-01,  4.529e-01, -0.000e+00, -0.000e+00,
        -1.946e-01, -5.223e-01, -0.000e+00,  0.000e+00, -4.891e-01]),
 'beta': array([ 0.0217, -0.0143, -0.0176, -0.0035,  0.0036,  0.006 ])}

In [14]:
m2 = Prophet().fit(df, init=m_params)

INFO:cmdstanpy:Chain [1] start processing
INFO:cmdstanpy:Chain [1] done processing


In [16]:
get_stan_init(m2)

{'k': -2.0216,
 'm': 0.57,
 'sigma_obs': 0.0372,
 'delta': array([ 0.    , -0.    ,  0.    ,  0.2229,  1.4402,  0.706 , -0.    ,
        -0.    , -0.    , -0.    , -0.    ,  0.535 ,  0.0016,  0.    ,
         0.    , -0.    ,  0.2122,  0.    , -0.    , -0.0242, -0.2163,
         0.    ,  0.    ,  0.158 ,  0.0817]),
 'beta': array([-0.0058,  0.0838,  0.0347, -0.0196,  0.0073, -0.0054, -0.0003,
         0.0063, -0.0163,  0.0033, -0.0006,  0.0073,  0.0039, -0.0011,
        -0.0038, -0.0015,  0.0031, -0.0056, -0.0003,  0.0032, -0.0319,
         0.0279,  0.0278,  0.007 , -0.004 , -0.009 ])}

In [81]:
def get_initial_values(df: pd.DataFrame, m: Prophet):
    history = df[df['y'].notnull()].copy()
    history_dates = pd.to_datetime(pd.Series(df['ds'].unique(), name='ds')).sort_values()

    history = m.setup_dataframe(history, initialize_scales=True)
    m.history = history
    m.set_auto_seasonalities()
    seasonal_features, prior_scales, component_cols, modes = (
        m.make_all_seasonality_features(history))
    train_component_cols = component_cols
    component_modes = modes
    fit_kwargs = {}

    m.set_changepoints()

    trend_indicator = {'linear': 0, 'logistic': 1, 'flat': 2}

    dat = {
        'T': history.shape[0],
        'K': seasonal_features.shape[1],
        'S': len(m.changepoints_t),
        'y': history['y_scaled'],
        't': history['t'],
        't_change': m.changepoints_t,
        'X': seasonal_features,
        'sigmas': prior_scales,
        'tau': m.changepoint_prior_scale,
        'trend_indicator': trend_indicator[m.growth],
        's_a': component_cols['additive_terms'],
        's_m': component_cols['multiplicative_terms'],
    }

    if m.growth == 'linear':
        dat['cap'] = np.zeros(m.history.shape[0])
        kinit = m.linear_growth_init(history)
    elif m.growth == 'flat':
        dat['cap'] = np.zeros(m.history.shape[0])
        kinit = m.flat_growth_init(history)
    else:
        dat['cap'] = history['cap_scaled']
        kinit = m.logistic_growth_init(history)

    stan_inits = {
        'k': kinit[0],
        'm': kinit[1],
        'delta': np.zeros(len(m.changepoints_t)),
        'beta': np.zeros(seasonal_features.shape[1]),
        'sigma_obs': 1,
    }
    return dat, stan_inits

In [84]:
from prophet.models import CmdStanPyBackend

In [85]:
mstan = CmdStanPyBackend()

In [86]:
previous_df = DATA.iloc[:500]
df = DATA.iloc[:510]
m = Prophet()

In [87]:
dat, stan_init = get_initial_values(df, m)

In [90]:
cmdstanpy_inits, cmdstanpy_data = mstan.prepare_data(stan_init, dat)

In [93]:
cmdstanpy_data

{'T': 510,
 'S': 25,
 'K': 26,
 'tau': 0.05,
 'trend_indicator': 0,
 'y': [0.5307510759405802,
  0.4724420380397057,
  0.43037623212550324,
  0.4442593363876163,
  0.45855893377759266,
  0.4430098570040261,
  0.40038872691933913,
  0.3913647091489657,
  0.41093988615854504,
  0.3848396501457726,
  0.3734555046508399,
  0.35915590726086355,
  0.3722060252672497,
  0.3652644731361932,
  0.3762321255032625,
  0.37498264611967236,
  0.3803970567818964,
  0.37859225322782175,
  0.392753019575177,
  0.4166319589060114,
  0.4360683048729696,
  0.4430098570040261,
  0.43870609468277105,
  0.4420380397056782,
  0.4588365958628349,
  0.4450923226433431,
  0.4595307510759406,
  0.4474524503679022,
  0.43537414965986393,
  0.4317645425517146,
  0.42718311814521726,
  0.43315285297792583,
  0.43690129112869636,
  0.44051089823684575,
  0.4466194641121755,
  0.43690129112869636,
  0.42995973899763984,
  0.42773844231570174,
  0.4264889629321116,
  0.39219769540469246,
  0.3899763987227544,
  0.40413

In [44]:
m2 = Prophet().fit(previous_df)

INFO:cmdstanpy:Chain [1] start processing
INFO:cmdstanpy:Chain [1] done processing


In [94]:
m2_params = get_stan_init(m2)

In [77]:
# m2_params['delta'] = m2_params['delta'] + np.random.normal(0, 0.01, size=m2_params['delta'].shape)
# m2_params['sigma_obs'] = 1
# m2_params['delta'] = np.zeros(m2_params['delta'].shape)
# m2_params['beta'] = np.zeros(m2_params['beta'].shape)
# m2_params['k'] = 0.3480494238511731
# m2_params['m'] = 0.5307510759405802

In [78]:
m3 = Prophet()

In [79]:
get_initial_values(df, m3)

{'k': 0.3480494238511731,
 'm': 0.5307510759405802,
 'delta': array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0.]),
 'beta': array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0.]),
 'sigma_obs': 1}

In [98]:
orig_dat, orig_init = get_initial_values(df, Prophet())

In [102]:
warm_inits, _ = mstan.prepare_data(m2_params, orig_dat)

In [103]:
stan_init, stan_data = mstan.prepare_data(orig_init, orig_dat)
args = dict(
    data=stan_data,
    inits=stan_init,
    algorithm='Newton' if stan_data['T'] < 100 else 'LBFGS',
    iter=int(1e4),
)
args.update({"inits": warm_inits})

In [149]:
fitted = Prophet().fit(previous_df)

INFO:cmdstanpy:Chain [1] start processing
INFO:cmdstanpy:Chain [1] done processing


In [153]:
fitted.params['beta']

array([[ 0.0216569 , -0.0142593 , -0.0176021 , -0.00349647,  0.00359015,
         0.00597893]])

In [122]:
orig_init

{'k': 0.3480494238511731,
 'm': 0.5307510759405802,
 'delta': array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0.]),
 'beta': array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0.]),
 'sigma_obs': 1}

In [132]:
m2_params

{'k': -0.9754,
 'm': 0.4568,
 'sigma_obs': 0.0501,
 'delta': array([ 0.000e+00,  0.000e+00,  0.000e+00,  3.765e-01,  7.350e-01,
         3.893e-01,  2.000e-04,  0.000e+00, -0.000e+00, -2.778e-01,
        -3.480e-01, -0.000e+00,  0.000e+00,  0.000e+00, -0.000e+00,
         5.647e-01,  9.929e-01,  4.529e-01, -0.000e+00, -0.000e+00,
        -1.946e-01, -5.223e-01, -0.000e+00,  0.000e+00, -4.891e-01]),
 'beta': array([ 0.0217, -0.0143, -0.0176, -0.0035,  0.0036,  0.006 ])}

In [144]:
m2_params_adj = {k: v for k, v in orig_init.items()}
# m2_params_adj['sigma_obs'] = 1

In [145]:
m2_params_adj['beta'] = np.array([ 0.0217, -0.0143, -0.0176, -0.0035,  0.0036,  0.006 ])

In [146]:
m4 = Prophet().fit(df, init=m2_params_adj)

INFO:cmdstanpy:Chain [1] start processing
INFO:cmdstanpy:Chain [1] done processing
ERROR:cmdstanpy:Chain [1] error: error during processing Stale NFS file handle
Optimization terminated abnormally. Falling back to Newton.
INFO:cmdstanpy:Chain [1] start processing
INFO:cmdstanpy:Chain [1] done processing
ERROR:cmdstanpy:Chain [1] error: error during processing Stale NFS file handle


RuntimeError: Error during optimization: console log output:

method = optimize
  optimize
    algorithm = newton
      newton
    iter = 10000
    save_iterations = 0 (Default)
id = 0 (Default)
data
  file = /var/folders/3x/ypx818511wsdnxc3knn7yy1r0000gn/T/tmpydjsgmb7/ud413oe8.json
init = /var/folders/3x/ypx818511wsdnxc3knn7yy1r0000gn/T/tmpydjsgmb7/meoqs7xx.json
random
  seed = 6113
output
  file = /var/folders/3x/ypx818511wsdnxc3knn7yy1r0000gn/T/tmpydjsgmb7/prophet_model-20230114225704.csv
  diagnostic_file =  (Default)
  refresh = 100 (Default)
  sig_figs = -1 (Default)
  profile_file = profile.csv (Default)

Rejecting initial value:
  Error evaluating the log probability at the initial value.
Exception: normal_id_glm_lpdf: Matrix of independent variables is inf, but must be finite! (in '/Users/cuong/projects/prophet/python/stan/prophet.stan', line 137, column 2 to line 142, column 4)
Rejecting initial value:
  Error evaluating the log probability at the initial value.
Exception: no

In [130]:
get_stan_init(m4)

{'k': -1.9736,
 'm': 0.5841,
 'sigma_obs': 0.0371,
 'delta': array([ 0.    , -0.    , -0.    ,  0.0533,  1.3097,  0.9564, -0.    ,
        -0.    , -0.    , -0.    ,  0.    ,  0.5909,  0.    , -0.    ,
        -0.    ,  0.    ,  0.174 ,  0.    , -0.    , -0.0845, -0.2968,
         0.    , -0.    ,  0.3614,  0.0362]),
 'beta': array([-0.0041,  0.089 ,  0.0333, -0.0196,  0.0072, -0.0058, -0.0001,
         0.0063, -0.0166,  0.0034, -0.0007,  0.0072,  0.0039, -0.0012,
        -0.0037, -0.0015,  0.0031, -0.0056, -0.0004,  0.0032, -0.0153,
         0.0147,  0.0134,  0.0037, -0.0017, -0.0043])}